Loading the Chat Model or LLM + Embeddign generations

In [71]:
from langchain.chat_models import ChatOllama # import chat model not llm, both are different
from langchain_core.messages import HumanMessage,SystemMessage # allows to define which is system message (context for human message) and which is human message
import os
from dotenv import load_dotenv
from langchain.embeddings import OllamaEmbeddings # embeddings models help convert data into vector embeddings

In [72]:
load_dotenv()
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")

In [73]:
llm =  ChatOllama(model='llama3.2') # load the chat model
embedding_model = OllamaEmbeddings(model="nomic-embed-text") # load the embedding model

In [74]:
vector = embedding_model.embed_query("What is LangChain?") # embedding process / function
print(vector[:5])

[-0.29047149419784546, 0.7891371846199036, -2.741931676864624, -1.7968902587890625, 0.7434597015380859]


In [75]:
messages =  messages = [ 
    SystemMessage("Translate the following from English into Hindi. Just give the translation straightforward Noting else"),
    HumanMessage("Context"),
] # define human and system messages

In [76]:
response = llm.invoke(messages) # call the chat model.
print(response.content)

संदर्भ


How to make vector databases using chromadb

In [77]:
import chromadb # import the chromadb library

In [78]:
chroma_client = chromadb.PersistentClient(path="./chroma_db") # store the db on the disk

In [79]:
collection = chroma_client.get_or_create_collection(name="my_collection") # create a collection. Collection are like sql tables but for vector databases

In [80]:
collection.add(
    ids=["doc1", "doc2"],
    metadatas=[{"source": "article"}, {"source": "blog"}],
    documents=["LangChain is a framework for LLM apps.", "Vector databases store embeddings efficiently."]
) # here we add some sample docs aling with metadata to pur collection

Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2


In [81]:
results = collection.query(
    query_texts=["What is databases"],
    n_results=1
) # Now we query and pritn teh results.

print(results)

{'ids': [['doc2']], 'embeddings': None, 'documents': [['Vector databases store embeddings efficiently.']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'blog'}]], 'distances': [[1.1538174968333408]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


Using ChromaDB with LangChain

In [82]:
from langchain.vectorstores import Chroma # immport chroma from langchain


In [83]:
embedding_model2 = OllamaEmbeddings(model="nomic-embed-text") # make and embedding model

In [84]:
vector_db = Chroma(persist_directory="./chroma_db2", embedding_function=embedding_model) # create a vector store

In [85]:
texts = ["LangChain is a powerful LLM framework.", "ChromaDB helps store embeddings."]
vector_db.add_texts(texts) # add some texts  /docs

['96bd2af3-79a7-466c-95ca-c092c8b8f70a',
 'dd5b8129-7293-467a-8f02-778a938e8cf9']

In [86]:
query = "what is used to save vector data ?"
results = vector_db.similarity_search(query, k=1)

print(results)

[Document(metadata={}, page_content='ChromaDB helps store embeddings.')]


 Storing Large Documents (Text Splitting)

In [87]:
from langchain.text_splitter import RecursiveCharacterTextSplitter # import the text splitter

In [88]:
text = "LangChain is a powerful tool for building applications with LLMs. It provides integrations with vector databases, retrieval mechanisms, and tools like ChromaDB."

In [89]:
# Split text
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
chunks = splitter.split_text(text)
print(chunks)

['LangChain is a powerful tool for building', 'building applications with LLMs. It provides', 'provides integrations with vector databases,', 'retrieval mechanisms, and tools like ChromaDB.']


Using retrivers - basic, max marginal relevance, multiquery

In [117]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# Load embeddings & ChromaDB
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
vector_db = Chroma(persist_directory="./chroma_db2", embedding_function=embedding_model)

# Create a retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})  # Fetch top 2 relevant docs

# Load LLM
llm = ChatOllama(model="llama3.2")

# Create a Retrieval-Augmented QA chain
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# Ask a question
query = "What is a vector database?"
response = qa_chain.run(query)

print(response)  # This should use retrieved docs + LLM to generate an answer


I don't know. I can provide some general information about vector databases, though.

A vector database is a type of data storage system designed to efficiently manage and query large collections of dense vectors in high-dimensional spaces. Vector databases are often used for applications such as recommendation systems, search, and clustering.

In the context of ChromaDB, it's likely that vector databases are being utilized to store and manage embeddings, which can be used for various purposes like facial recognition, object detection, or natural language processing.
